# HW 2

## Imports

In [ ]:
import nbtools
nbtools.setup_nb()

In [ ]:
import itertools
from astropy import units
import sympy
import pandas
from scipy import integrate
import numpy
import plotly.express as px
from plotly import graph_objects as go

from sympy.diffgeom import Manifold, Patch
from pystein import coords, metric, curvature, geodesic
from pystein.utilities import tensor_pow as tpow, full_simplify, boundary_filter

In [ ]:
show_plots = True

## Utilities

In [ ]:
import plotly.colors

def get_continuous_color(colorscale, intermed):
    """
    Plotly continuous colorscales assign colors to the range [0, 1]. This function computes the intermediate
    color for any value in that range.

    Plotly doesn't make the colorscales directly accessible in a common format.
    Some are ready to use:
    
        colorscale = plotly.colors.PLOTLY_SCALES["Greens"]

    Others are just swatches that need to be constructed into a colorscale:

        viridis_colors, scale = plotly.colors.convert_colors_to_same_type(plotly.colors.sequential.Viridis)
        colorscale = plotly.colors.make_colorscale(viridis_colors, scale=scale)

    :param colorscale: A plotly continuous colorscale defined with RGB string colors.
    :param intermed: value in the range [0, 1]
    :return: color in rgb string format
    :rtype: str
    """
    if len(colorscale) < 1:
        raise ValueError("colorscale must have at least one color")

    if intermed <= 0 or len(colorscale) == 1:
        return colorscale[0][1]
    if intermed >= 1:
        return colorscale[-1][1]

    for cutoff, color in colorscale:
        if intermed > cutoff:
            low_cutoff, low_color = cutoff, color
        else:
            high_cutoff, high_color = cutoff, color
            break

    # noinspection PyUnboundLocalVariable
    return plotly.colors.find_intermediate_color(
        lowcolor=low_color, highcolor=high_color,
        intermed=((intermed - low_cutoff) / (high_cutoff - low_cutoff)),
        colortype="rgb")

## Exercises

### B2 - Rindler Coordinates

#### Setup Metric

In [ ]:
M = Manifold('M', dim=2)
P = Patch('origin', M)

In [ ]:
rho, eta = sympy.symbols('rho eta', nonnegative=False)
cs = coords.CoordSystem('cartesian', P, [eta, rho])
deta, drho = cs.base_oneforms()
ds2 = - rho ** 2 * tpow(deta, 2) + tpow(drho, 2)
g_rind = metric.Metric(twoform=ds2)
g_rind

#### Compute curvature components

In [ ]:
crs, rmns, rcs = curvature.compute_components(g_rind)

In [ ]:
curvature.display_components(crs)

In [ ]:
curvature.display_components(rmns)

In [ ]:
curvature.display_components(rcs)

### B3

In [ ]:
rhos = numpy.arange(1/numpy.e**2, 10, 0.01)
consts = numpy.arange(-2, 12, 1.0)
c_norm = lambda c: (c + 2) / 14
pos_etas = [+numpy.log(rhos) + c for c in consts]
neg_etas = [-numpy.log(rhos) + c for c in consts]

In [ ]:
viridis_colors, _ = plotly.colors.convert_colors_to_same_type(plotly.colors.sequential.Plasma)
colorscale = plotly.colors.make_colorscale(viridis_colors)

In [ ]:
fig = go.Figure()
for c, n, p in zip(consts, neg_etas, pos_etas):
    color = get_continuous_color(colorscale, intermed=c_norm(c))
    fig.add_trace(go.Scatter(x=rhos, y=p, line=dict(color=color, dash='solid')))
    fig.add_trace(go.Scatter(x=rhos, y=n, line=dict(color='rgba'+color[3:-1]+', 0.6)', dash='dot')))

fig.update_layout(yaxis_range=[0,10],
                  xaxis_range=[0,10],
                  width=700,
                  height=700,
                  showlegend=False,
                  title_text=r'$\text{Null Geodesics in 2D Minkowski }(\eta, \rho)$', 
                  title_x=0.5,
                  xaxis_title=r'$\rho$',
                  yaxis_title=r'$\eta$')
fig.show()

In [ ]:
# fig.write_image('/Users/jim/repos/tex/homework/courses/PHYS510 - GR I/figures/fig-b3.pdf')

### B4

In [ ]:
xs = numpy.arange(0.0, 1.0, 0.01)
ts_p = numpy.arange(0.0, 1.0, 0.01)
ts_m = numpy.arange(-1.0, 0.0, 0.01)

# Const Eta
etas = numpy.arange(-3, 3, .25)
eta_norm = lambda e: abs(e) / 3
ts_eta_const = [xs * numpy.tanh(e) for e in etas]

# Const rho
rhos = numpy.arange(0.0, 1.0, 0.1)
rho_norm = lambda r: 1 - r
xs_rho_const_p = [numpy.sqrt(ts_p ** 2 + r ** 2) for r in rhos]
xs_rho_const_m = [numpy.sqrt(ts_m ** 2 + r ** 2) for r in rhos]

In [ ]:
_tmp_const_eta, _ = plotly.colors.convert_colors_to_same_type(plotly.colors.sequential.Blues)
_tmp_const_rho, _ = plotly.colors.convert_colors_to_same_type(plotly.colors.sequential.OrRd)
colorscale_eta = plotly.colors.make_colorscale(_tmp_const_eta)
colorscale_rho = plotly.colors.make_colorscale(_tmp_const_rho)

In [ ]:
fig = go.Figure()

# Plot const rho
for r, n, p in zip(rhos, xs_rho_const_m, xs_rho_const_p):
    color_r = get_continuous_color(colorscale_rho, intermed=0.5*rho_norm(r) + 0.5)
#     print(r, rho_norm(r), 0.5*rho_norm(r) + 0.5)
#     print(color_r)
    fig.add_trace(go.Scatter(x=n, y=ts_m, line=dict(color=color_r, dash='solid')))
    fig.add_trace(go.Scatter(x=-n, y=ts_m, line=dict(color=color_r, dash='solid')))
    fig.add_trace(go.Scatter(x=p, y=ts_p, line=dict(color=color_r, dash='solid')))
    fig.add_trace(go.Scatter(x=-p, y=ts_p, line=dict(color=color_r, dash='solid')))
    
for e, t in zip(etas, ts_eta_const):
    color_e = get_continuous_color(colorscale_eta, intermed=0.5*eta_norm(e) + 0.5)
#     print(e, eta_norm(e), color)
    fig.add_trace(go.Scatter(x=xs, y=t, line=dict(color=color_e, dash='solid')))
    fig.add_trace(go.Scatter(x=-xs, y=-t, line=dict(color=color_e, dash='solid')))

    
fig.update_layout(yaxis_range=[-1,1],
                  xaxis_range=[-1,1],
                  width=550,
                  height=550,
                  showlegend=False,
                  title_text=r'$\text{Rindler Wedge in 2D Minkowski Metric }(t, x)$', 
                  title_x=0.5,
                  xaxis_title=r'$x$',
                  yaxis_title=r'$t$')
fig.show()

In [ ]:
# fig.write_image('/Users/jim/repos/tex/homework/courses/PHYS510 - GR I/figures/fig-b4.pdf')

### B5

Simplify Expression for $\lambda_2$

In [ ]:
d, g, z = sympy.symbols('d g z')

In [ ]:
x_arg_alt = sympy.Rational(1, 2) * (z + 1) + 1 / (z + 1)
sympy.simplify(x_arg_alt)

In [ ]:
x_arg_z = sympy.simplify((z**2 * (z + 2)**2 + 2 * (z + 1)**2) / (2 * (z + 1)**2))
x_arg_z

In [ ]:
x_arg_ln_z = x_arg_z + sympy.sqrt(x_arg_z ** 2 - 1)
x_arg_ln_z

In [ ]:
lam_2_z_alt = sympy.ln(x_arg_ln_z)
lam_2_z_alt

In [ ]:
(sympy.series(lam_2_z_alt, x=z, n=3).subs([(z, d * g)]) / g).doit()

### B8

Make visual of $\rho_{\mathrm{obs}}(\tau)$

In [ ]:
# gs = 1 + numpy.arange(-1 + 0.1, 50, 0.1)
gs = 10.0 ** numpy.arange(-1.2, 1.2, 0.1)
# gs = numpy.logspace(-2, 2, int(4/0.1))
rho_f = 1
P_f = 1

g_norm = lambda g: (numpy.log(g) + 1.2) / (1.2 - -1.2) - 0.1

taus_ = []
rhos_ = []

for g in gs:
    gtau_bound = numpy.log(1 + numpy.sqrt(2))
    tau_bound = gtau_bound / g

    taus = numpy.arange(0, 1.0, 0.01)
    vs = numpy.sinh(g * taus)
    gammas_sq = 1 / (1 - vs ** 2)
    rho_obs = gammas_sq * (rho_f + vs ** 2 * P_f)
    rho_obs[numpy.where(taus > tau_bound)] = numpy.nan
    
    taus_.append(taus)
    rhos_.append(rho_obs)

In [ ]:
_tmp_const_g, _ = plotly.colors.convert_colors_to_same_type(plotly.colors.sequential.Agsunset)
colorscale_g = plotly.colors.make_colorscale(_tmp_const_g)

In [ ]:
fig = go.Figure()

for g, ts, rs in zip(gs, taus_, rhos_):
    color_g = get_continuous_color(colorscale_g, g_norm(g))
    fig.add_trace(go.Scatter(x=ts, 
                             y=rs, 
                             line=dict(color=color_g, 
                                       dash='solid'),
                 ))
    fig.add_trace(go.Scatter(x=[-1], 
                             y=[-1], 
                             marker=dict(
                                size=16,
                                cmax=39,
                                cmin=0,
                                color=color_g,
                                colorbar=dict(
                                    title="Field g"
                                ),
                                colorscale="Agsunset"
                            ),
                             mode='markers'
                 ))

    
fig.update_layout(
    yaxis_range=[0.95,2.05],
    xaxis_range=[-.05,1],
    width=700,
    height=550,
    showlegend=False,
    title_text=r'$\text{Measured energy density vs proper time}$', 
    title_x=0.5,
    xaxis_title=r'$\tau$',
    yaxis_title=r'$\rho_{\mathrm{obs}}/\rho_{\mathrm{f}}$',
)
fig.show()

In [ ]:
fig.write_image('/Users/jim/repos/tex/homework/courses/PHYS510 - GR I/figures/fig-b8.pdf')

## C - Problem

### C1

In [ ]:
M = Manifold('M', dim=2)
P = Patch('origin', M)
t, h, g, c = sympy.symbols('t h g c', nonnegative=False)

cs = coords.CoordSystem('cartesian', P, [t, h])
dt, dh = cs.base_oneforms()

ds2 = - (1 + g * h / c ** 2) ** 2 * tpow(dt, 2) + tpow(dh, 2)
g_c1 = metric.Metric(twoform=ds2)
g_c1

In [ ]:
crs, rmns, rcs = curvature.compute_components(g_c1)

In [ ]:
curvature.display_components(crs)

In [ ]:
sympy.Derivative(c**2/(c**2 + g * h), h).doit()